In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [43]:
bookdata = pd.read_csv('/opt/ml/finalproject/book-data/merge_data_re.csv')
songdata = pd.read_csv('/opt/ml/finalproject/kakaomusic/songs2.csv')

In [4]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
# tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

> # KR-SBERT 모델 불러오기

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 
sentences = ['잠이 옵니다', '졸음이 옵니다', '기차가 옵니다']
vectors = model.encode(sentences) # encode sentences into vectors
# similarities = util.cos_sim(vectors, vectors) # compute similarity between sentence vectors
# print(similarities)
# tensor([[1.0000, 0.6577, 0.2732],
#         [0.6577, 1.0000, 0.2730],
#         [0.2732, 0.2730, 1.0000]])

In [6]:
# len(vectors)
len(vectors[0])

768

> # 책 샘플 1000개

In [10]:
songdata.head(5)

,song_id,song_title,artist,song_genre,song_genre_id,album_id,playlist_tag,playlist_title,lyric
0,579571,Bezerk (Feat. A$AP Ferg),Big Sean,랩/힙합,GN1200,10321695,3105|40027091|5년동안|8281|HipHop|Hot100|Party|hi...,외국 힙합 띵곡 모음|듣기만해도 3대500할 것 같은 힙합!|턴업!! 춤추고 싶게 ...,"Yeah, yeah (Hit-Boy)| Woah yeah, yeah| Sean, w..."
1,579698,Take U There (Feat. Kiesza) (Missy Elliott Rem...,Jack U,일렉트로니카,GN1100,2307764,2019UMF|Club|EDM|Electronic|Festival|GZ|HipHop...,#어디가서 힙알못 소리는 듣지말자 (해외)|#11 foreign swag songs...,You show me there's something more to us|Than ...
2,580131,King Of My Heart,Taylor Swift,POP,GN0900,10091197,EDM|Hook|Pop|RnB|electronica|감성|개강|고등학생|공연|국외영...,해외에서 들으면 느낌 쩌는 취향저격 EDM & PoP|여행 갈때 이 곡과 떠나요~!...,I'm perfectly fine I live on my own|I made up ...
3,581120,Not My Friend (Feat. PLUTO),땡우,인디음악,GN0500,2668003,2016년도1분기|MC|OwenOvadoz|겨울|결산|공연|국내외|국내힙합|기분전환...,"대학교 축제 기간! 주점에서 플레이하기 좋은 알앤비, 힙합|힙합 틀어주세여 현기증난...",i came from LEVEL 0|대놓고 면전에다|욕하는 놈 어딨겠냐만|다 써져있...
4,581409,Pretend,Mali-Koa Hood,POP,GN0900,10230894,Chill|Pop|감동|감미로운|감성|기분전환|기분좋은|나른한노래|달닻|독서할때|드...,힘든 청춘에게 바치는 랩소디|잔잔한 바람과 차분하게 듣기좋은 음악|[POP/KPOP...,NaN


In [11]:
# minyoung.head(5)
# songdata.columns = ['song_id','song_title','artist','song_genre']


,곡ID,제목,가수,장르,장르ID,plylst_태그,plylst_제목,plylst_태그길이,가사
0,144663,밤편지,['아이유'],['발라드'],['GN0100'],"['독보적', 'Zzz', '눈오는날', 'electronica', '사지방', '...","['마음을 위로해주는 최신 인기 발라드', '효리네 민박 cafe_ 잔잔한 플레이리...",1941.0,이 밤 그날의 반딧불을 |당신의 창 가까이 보낼게요|음 사랑한다는 말 이에요||나 ...
1,205179,꿀맛,['정미애'],['성인가요'],['GN0700'],"['주인공', '눈오는날', '용기', '안동역', 'Song', '방', '이찬원...","['나른한 오후 따듯한 커피 한잔', '주말 하루종일 뒹굴뒹굴 듣고 싶은 노래(부제...",1804.0,이리 보아도 내 사랑|얼씨구 좋구나 좋다 좋아|저리 보아도 내 사랑 꿀맛 같은 내 ...
2,116573,안아줘,['정준일'],"['인디음악', '록/메탈']","['GN0500', 'GN0600']","['가수', '눈오는날', 'Song', '방', '울컥', '북카페', '소울',...","['걱정', '고민', '스트레스 치료용 음악 TOP 10', '새벽 1시에 들으면...",1415.0,서러운 맘을 못 이겨|잠 못 들던 어둔 밤을 또 견디고|내 절망관 상관없이|무심하게...
3,246531,"모든 날, 모든 순간 (Every day, Every Moment)",['폴킴'],"['발라드', 'OST']","['GN0100', 'GN1500']","['그겨울', '울적한', '눈오는날', '5월', '용기', '피아노음악', '두...","['이별 후 공감할 노래 그가 생각나는 밤', '커피한잔에 어울리는 감성충전 노래모...",1386.0,네가 없이 웃을 수 있을까|생각만 해도 눈물이나|힘든 시간 날 지켜준 사람|이제는 ...
4,224921,첫눈처럼 너에게 가겠다,['에일리'],"['발라드', 'OST']","['GN0100', 'GN1500']","['그겨울', '그룹', '눈오는날', 'Song', '음악감독', '울컥', '대...","['드라마 OST 모음', '그냥 좋은 노래들', '', '도깨비 OST...', ...",1371.0,널 품기 전 알지 못했다|내 머문 세상 이토록 찬란한 것을||작은 숨결로 닿은 사람...


In [54]:
sample_books = bookdata
book_features = ['name','index','intro','review']

In [55]:
features_book_sample = sample_books[book_features]
features_book_sample.sample(5)

,name,index,intro,review
2353,박물지,해설\n\n지은이에 대해\n\n\n\n1권\n\n지리략(地理略)\n\n땅[地]\n\...,전 세계의 다양한 분야의 고전을 만나본다!\n\n\n\n『지만지 고전천줄』은 전 세...,세상의 온갖 기괴한 이야기를 모아놓은 보물창고 같은 책. 인간의 삶에 대한 모든 것...
3185,문명 1,제1막 지상 낙원\n\n제2막 제3의 눈,인류 문명의 끝에서 새로운 이야기가 시작된다!\n\n프랑스 25만 부 판매 베스트셀...,NaN
6949,로리타,NaN,"부녀간의 부도덕한 관계를 다룬 러시아 태생 작가의 소설. 인간성 해방, 자유로움을...",머리말 \n\n[로리타]를 소설로 써야겠다는 충동이 처음 나에게 생겼을 때는 내가 ...
8271,클링조르를 찾아서 1,프롤로그\n\n\n\n1부\n\n[이야기의 운동법칙]\n\n1법칙: 모든 이야기는 ...,멕시코의 신예작가 호르헤 볼피의 소설 『클링조르를 찾아서』제1권. 나치의 원자탄 프...,"라틴문학의 영역을 한 단계 높인 최고의 교양소설!\n\n멕시코의 작가, 호르헤 볼피..."
1508,소설의 신 : 너를 읽는 이야기 (下),제 4화 마음이 움직인 뒤에 ...,너 때문에 더 이상 쓸 수 없어. 친구에게 소설의 가치를 부정당한 나루세. 그녀는 ...,하시모토 칸나 X 사토 타이키(EXILE / FANTASTICS) 주연의 영화 원작...


In [56]:
f_book_sample_ll = features_book_sample.values.tolist()
# f_book_sample_ll[0]

> ## 책임베딩 만들기

In [49]:
ssss = sample_books.iloc[:10]

In [53]:
len(sample_books)

9991

In [57]:
import re
bookembeddings = []
device = torch.device('cuda')
for row in tqdm(f_book_sample_ll):
    # print(row[2])
    sentence_ll = []
    vector_ll = []
    for column in row:
        if not pd.isna(column):
            temp = re.split("[.'\n''\t']",column)
        # print(temp)
        for i in temp:
            if i != '' and i != ' ':
                sentence_ll.append(i)
    # print(sentence_ll)
        # break
    # break
    vectorsum = 0
    for sentence in sentence_ll:
        vectorsum += model.encode(sentence)
    vectorsum = vectorsum/len(sentence_ll)
    bookembeddings.append(vectorsum)
    # print(bookembeddings)
    # break
            
# ssss['embedding'] = bookembeddings
# ssss
sample_books['embedding'] = bookembeddings
sample_books
sample_books.to_csv('bookfullemb.csv')


100%|██████████| 9991/9991 [2:35:20<00:00,  1.07it/s]   


In [29]:
sample_books.to_csv('booksample1000emb.csv')


In [58]:
sample_books.iloc[0].embedding

array([-4.14532483e-01, -5.60343683e-01, -5.45689225e-01,  1.75572664e-01,
        5.32399267e-02,  1.06446221e-01,  3.15445453e-01, -3.76039237e-01,
       -1.95934013e-01, -4.90194768e-01, -1.89323291e-01,  2.66736865e-01,
        2.78133810e-01,  1.26488972e-02, -3.35877657e-01,  9.24514085e-02,
        5.96565902e-02, -5.55381000e-01, -4.05565128e-02,  7.28814542e-01,
        1.99253380e-01, -2.22351164e-01,  5.90990335e-02,  4.13899601e-01,
        1.51753977e-01, -9.76599678e-02, -3.05438399e-01,  4.06520039e-01,
       -9.79398936e-02,  8.13121572e-02,  2.71109074e-01, -1.11870646e+00,
       -4.98399101e-02, -3.19358557e-01,  2.68519193e-01, -3.67302537e-01,
        2.36561418e-01,  4.20430481e-01,  5.41449664e-03, -4.08731729e-01,
       -6.85517788e-01,  4.51246798e-01, -9.71034840e-02, -3.06614965e-01,
       -6.05200887e-01,  2.55757689e-01,  4.25299734e-01,  4.09091502e-01,
        8.40960443e-02,  1.25741974e-01,  6.06436357e-02, -3.47623117e-02,
       -2.58192718e-01, -

> # 노래 샘플 1000개

In [12]:
songdata.sample(5)

,song_id,song_title,artist,song_genre,song_genre_id,album_id,playlist_tag,playlist_title,lyric
9496,704155,꽃길 (With ZISU),새봄 (saevom),인디음악,GN0500,10148126,3월|4월|5월|감성|개화|계절|고백|기분전환|기쁨|꿀잠|나들이|나만알고싶은|눈|달...,프로듀싱전문 아티스트 새봄 전곡모음입니다.|'벚꽃이 핀다면 찾아주세요' 봄맞이 리스...,음 눈부시게 빛난 오월의 첫날|그 봄을 기억하니|음 그때부터 너와 함께 있으면|하루...
14265,655317,어쩌다..,윤건,R&B/Soul,GN0400,33910,2000|2000년대|2시|7080|8090|LOVE|Top3|dj딘|soul|to...,기억속의 잔잔한 발라드|리메이크 & 드라마|토토가 덕분에 다시듣는 명곡들!|슬픔2|...,어쩌다 지나친 거릴 돌아보게 돼 |니가 있는 게 아닐텐데 맞을텐데 |어쩌다 오간 대...
18856,624848,Listen,Beyonce,OST,GN1500,345414,0515|2000|2000년|2000년대|2000년대명곡들|2000년대팝송|2010...,힘들고지칠때..힘이되는..노래~*^^*|나만의 팝송(애플)|최근들은 나의 리스트|쭉...,Listen to the song here in my heart|A melody I...
12821,373702,넌 할 수 있어,강산에,록/메탈,GN0600,3031,00년대|1990년대|2000년대|2000년초|30대|90년대|90년대노래|90년대...,아무것도 위로 되지 못할 때 위로가 되어준 음악|나는 위로가 필요하다|[시대별차트]...,후회하고 있다면 깨끗이 잊어버려 |가위로 오려낸 것처럼 다 지난 일이야 ||후회하지...
16465,558685,싸우지 말자,앤츠 (Ants),인디음악,GN0500,10087107,2017년|BEST_of_best|Folk|OST|Rock|ralalra|가요|가을...,"한 소절만 들으면 빠지는 노래 7|""데이트하기 좋은 휴일"" 이노래 들으며 러브힐링 ...",01 싸우지 말자 ||혼자 앉은 버스에서 한참을 생각했어|돌아서던 네 모습이 자꾸만...


In [13]:
sample_song = songdata
# song_features = ['song_genre','playlist_tag','playlist_title','lyric']
song_features = ['song_genre','playlist_title','lyric']



In [14]:
features_song_sample = sample_song[song_features]
features_song_sample.sample(5)

,song_genre,playlist_title,lyric
15336,인디음악,악동서울DJ 런쥔(NCT DREAM) 추천곡!|여름밤 특유의 공기가 주는 두근거림2...,그날 나는 많은 질문을 던졌었죠||서로 많은 이야기를 나눴어요|마음과 마음들 순간 ...
4444,록/메탈,하루종일 방에서 뒹굴거리며 듣는 음악들|E 너와 분위기잡을때 (사랑시리즈)|❤️ 카...,unk
16892,일렉트로니카,자신감 충전! 파워워킹하게 해주는 비트 좋은 음악들|겨울 끝!! 봄을 기다리며 듣는...,I'ma leave my home|I'ma do it all|I was lookin...
9084,POP,"골든 리스트! 편안하고 분위기있는 시간들:)2 (팝, 어쿠스틱, 컨트리)|로맨틱 크...",December come to me|I hope I can see|You not j...
23433,POP,온라인 과제와 맞짱까고 엉덩이 흔들어 재낄때 song| 차곡차곡 모아온 띵곡 모음|...,I know I fucked up I'm just a loser|Shouldn't ...


In [15]:
f_song_sample_ll = features_song_sample.values.tolist()


> ## 노래 임베딩 만들기

In [16]:
import re
device = torch.device('cuda')
model.to(device)
song_embeddings = []
for row in tqdm(f_song_sample_ll):
    sentence_ll = []
    vector_ll = []
    for column in row:
        if not pd.isna(column):
        # print(row[1].split('|'))
            temp = column.split('|')
        for i in temp:
            sentence_ll.append(i)
    vectorsum = np.zeros(768)
    # vectorsums = torch.Tensor(vectorsum)
    # vectorsums = vectorsum.to(device)  

    # sentence_lls = torch.Tensor(vectorsum)
    # sentence_lls.to(device)
    # print(sentence_ll)
    # break
    for sentence in sentence_ll:
        if i != '' and i != ' ':
            vectorsum += model.encode(sentence)
        # print(len(model.encode(sentence)))
        # break
    vectorsum = vectorsum/len(sentence_ll)
    # print(vectorsum[:5])
    # break
    # vectorsum = vectorsum.to('cpu')  

    song_embeddings.append(vectorsum)
    # break
# len(song_embeddings[0])
sample_song['embedding'] = song_embeddings
sample_song
sample_song.to_csv('fullsongemb.csv')

    

100%|██████████| 40000/40000 [23:11:15<00:00,  2.09s/it]    


In [43]:
sample_song.to_csv('songemb.csv')

> # inference

In [31]:
sample_song = sample_song[['id','song_id','song_title','embedding']]
sample_song.head(5)

,id,song_id,song_title,embedding
0,0,579571,Bezerk (Feat. A$AP Ferg),"[-0.37724633280791947, -0.5276857299000226, -0..."
1,1,579698,Take U There (Feat. Kiesza) (Missy Elliott Rem...,"[-0.22316226534425251, -0.6031812865647578, -0..."
2,2,580131,King Of My Heart,"[-0.3138808202412393, -0.6123100653474713, -0...."
3,3,581120,Not My Friend (Feat. PLUTO),"[-0.28439008424548096, -0.3766443343953982, 0...."
4,4,581409,Pretend,"[-0.17954782396554947, -0.20745742022991182, -..."


In [59]:
sample_song.iloc[0].embedding

array([-3.77246333e-01, -5.27685730e-01, -2.92555423e-01, -4.07986307e-01,
        2.70774517e-01,  3.68057551e-01,  1.52847703e-01, -4.74679719e-01,
        5.63018827e-02, -6.76311708e-02, -1.21791399e-01, -8.24156728e-02,
        7.43827700e-01,  4.83784309e-01, -4.68403771e-01,  1.27593511e-01,
        6.19301880e-02,  3.39114028e-02,  1.67491215e-01,  5.33108634e-02,
        1.19871201e-02,  2.36372473e-01, -1.50795266e-02, -1.84346588e-01,
       -2.57349270e-01, -3.89590076e-01, -7.73394403e-02, -3.03513771e-01,
        1.84912029e-01, -1.35172767e-01, -3.64937364e-01, -6.68883713e-02,
        2.92059408e-01,  2.63815065e-01,  5.39013909e-01, -3.10168695e-01,
       -9.67500646e-01, -1.81464099e-02,  3.93100837e-01, -8.93494626e-02,
       -8.44407884e-01, -4.61298813e-01,  2.04374593e-01,  1.15069180e-01,
       -6.49800960e-01,  1.38566834e-01,  3.72638712e-01,  6.71408951e-01,
        1.59421523e-01, -6.82826747e-01, -1.57658494e-01, -5.09749867e-02,
        8.86250835e-02,  

In [17]:
from numpy import dot
from numpy.linalg import norm

In [18]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [32]:
def return_answer(embedding):
    # embedding = model.encode(question)
    sample_song['score'] = sample_song.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    # return sample_song.loc[sample_song['score'].nlargest(3)]
    return sample_song.nlargest(15,'score')

In [36]:
books = pd.read_csv('/opt/ml/finalproject/bookfullemb.csv')
len(books)

9991

In [62]:
books = sample_books[['isbn','embedding']]
books = books.values.tolist()
# books

In [73]:
recommendation = pd.DataFrame(columns = {'id','song_id','bookisbn'})

In [74]:
[1]*15

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [80]:
recommendation = pd.DataFrame(columns = {'id','song_id','bookisbn'})

for book in tqdm(books):
    isbn = book[0]
    embedding = book[1]
    
    a= return_answer(embedding)
    a['bookisbn'] = [isbn]*15
    temp = a[['id','song_id','bookisbn']]
    recommendation = pd.concat([recommendation,temp], ignore_index=True)
    # recommendation.append(temp, ignore_index=True)
    # print(temp)

    # print(a[['id','song_id','bookisbn']])

    # print(embedding.dtype())
    # print(embedding)

    # print(a)
    # break
# print(recommendation)
recommendation.to_csv('full_recommendation.csv',index=False)

100%|██████████| 9991/9991 [3:33:01<00:00,  1.28s/it]  


In [81]:
len(recommendation)/15

9991.0

In [82]:
recommendation

,id,bookisbn,song_id
0,29113,9791191043723,180173
1,2746,9791191043723,547773
2,7556,9791191043723,491417
3,2162,9791191043723,701607
4,3539,9791191043723,700091
...,...,...,...
149860,35485,9791155160497,191072
149861,11955,9791155160497,674693
149862,39124,9791155160497,108184
149863,33838,9791155160497,1099


In [69]:
candidate_book = sample_books.sample(1)
print(candidate_book['name'])
candidaste_song = return_answer(candidate_book['embedding'].values[0])
print(candidaste_song[['song_title','artist','song_genre']])


5318    발푸르가의 진주 목걸이 1
Name: name, dtype: object
                                             song_title           artist  \
635                                          More Sugar           divin`   
6232                                 トトロ / Totoro (토토로)     Hisaishi Joe   
1189  The Greatest, Part 2 (Feat. Los, CHANGMO & Owe...  팔로알토 (Paloalto)   

     song_genre  
635    R&B/Soul  
6232        OST  
1189       랩/힙합  
